# 100本　第5章

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from dateutil.relativedelta import relativedelta

In [75]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

## ノック41:データの読み込みと整形

In [2]:
customer = pd.read_csv('customer_join.csv')
customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,ナイト,6000,通常,3.916667,4.0,6,1,1,2019-04-30,47


In [8]:
uselog_months = pd.read_csv('use_log_months.csv')
uselog_months.head()

,年月,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


In [9]:
year_months = list(uselog_months['年月'].unique())

In [13]:
uselog_months.loc[uselog_months['年月'] == year_months[1]]

,年月,customer_id,count
2991,201805,AS002855,5
2992,201805,AS009373,4
2993,201805,AS015233,7
2994,201805,AS015315,3
2995,201805,AS015739,5
...,...,...,...
6049,201805,TS993439,10
6050,201805,TS995299,7
6051,201805,TS999079,5
6052,201805,TS999231,4


In [40]:
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months['年月'] == year_months[i]]
    tmp.rename(columns={'count':'count_0'}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months['年月'] == year_months[i-1]]
    del tmp_before['年月']
    tmp_before.rename(columns={'count':'count_1'}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on='customer_id', how='left')
    uselog = pd.concat([uselog, tmp], ignore_index=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [41]:
uselog.head()

,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


## ノック42:退会前月の退会顧客データを作成

In [34]:
exit_customer = customer[customer['is_deleted'] == 1]
exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
exit_customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-04-30,23
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,デイタイム,7500,通常,3.0,3.0,3,3,0,2018-04-30,23
730,HD542886,XX,C01,M,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,1.0,1.0,1,1,0,2018-04-30,23
770,HD597545,XXXXX,C03,F,2016-06-01,2018-05-31,CA1,1,ナイト,6000,通常,3.5,3.5,4,3,1,2018-05-31,23
785,HI749296,XXXXX,C01,M,2016-06-01,2018-05-31,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-05-31,23


In [43]:
# for i in range(len(exit_customer)):
#     exit_customer['exit_date'][i] = exit_customer['end_date'][i] - relativedelta(months=1)
exit_customer['exit_date'] = exit_customer['end_date'].apply(lambda x: x - relativedelta(months=1))
exit_customer['年月'] = exit_customer['exit_date'].dt.strftime('%Y%m')
uselog['年月'] = uselog['年月'].astype(str)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [52]:
exit_uselog = pd.merge(uselog, exit_customer, on=['customer_id', '年月'], how='left')
print(len(exit_uselog))
exit_uselog.head()

33851


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [54]:
exit_uselog = exit_uselog.dropna(subset=['name'])
print(len(exit_uselog))
print(exit_uselog['customer_id'].nunique())
exit_uselog.head()

1104
1104


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30


## ノック43:継続顧客のデータを作成

In [56]:
conti_customer = customer[customer['is_deleted'] == 0]
conti_uselog = pd.merge(uselog, conti_customer, on='customer_id', how='left')
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=['name'])
print(len(conti_uselog))
# exit_customer['exit_date'] = 0
# conti_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])

33851
27422


In [57]:
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset='customer_id')
print(len(conti_uselog))
conti_uselog.head()

2842


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201809,IK166339,4,10.0,XXXX,C01,M,2017-04-01,NaN,CA1,...,オールタイム,10500.0,通常,5.916667,5.0,10.0,3.0,1.0,2019-04-30,24.0
1,201902,OA782311,5,8.0,XXXX,C01,M,2018-01-01,NaN,CA1,...,オールタイム,10500.0,通常,6.833333,6.5,10.0,3.0,1.0,2019-04-30,15.0
2,201809,HD642174,4,4.0,XXXX,C02,F,2017-02-01,NaN,CA1,...,デイタイム,7500.0,通常,5.333333,5.5,8.0,2.0,1.0,2019-04-30,26.0
3,201808,HI859532,5,5.0,XXX,C02,F,2016-03-01,NaN,CA1,...,デイタイム,7500.0,通常,5.000000,5.0,7.0,2.0,1.0,2019-04-30,37.0
4,201812,HD459783,1,3.0,XXXXX,C01,M,2015-09-01,NaN,CA1,...,オールタイム,10500.0,通常,4.416667,4.5,9.0,1.0,1.0,2019-04-30,43.0


In [60]:
conti_uselog.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2842 entries, 0 to 27274
Data columns (total 21 columns):
年月                   2842 non-null object
customer_id          2842 non-null object
count_0              2842 non-null int64
count_1              2637 non-null float64
name                 2842 non-null object
class                2842 non-null object
gender               2842 non-null object
start_date           2842 non-null object
end_date             0 non-null object
campaign_id          2842 non-null object
is_deleted           2842 non-null float64
class_name           2842 non-null object
price                2842 non-null float64
campaign_name        2842 non-null object
mean                 2842 non-null float64
median               2842 non-null float64
max                  2842 non-null float64
min                  2842 non-null float64
routine_flg          2842 non-null float64
calc_date            2842 non-null object
membership_period    2842 non-null float64
dtype

In [62]:
exit_uselog.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1104 entries, 19 to 30852
Data columns (total 22 columns):
年月                   1104 non-null object
customer_id          1104 non-null object
count_0              1104 non-null int64
count_1              1052 non-null float64
name                 1104 non-null object
class                1104 non-null object
gender               1104 non-null object
start_date           1104 non-null object
end_date             1104 non-null datetime64[ns]
campaign_id          1104 non-null object
is_deleted           1104 non-null float64
class_name           1104 non-null object
price                1104 non-null float64
campaign_name        1104 non-null object
mean                 1104 non-null float64
median               1104 non-null float64
max                  1104 non-null float64
min                  1104 non-null float64
routine_flg          1104 non-null float64
calc_date            1104 non-null object
membership_period    1104 non-null f

In [63]:
predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,calc_date,campaign_id,campaign_name,class,class_name,count_0,count_1,customer_id,end_date,exit_date,...,max,mean,median,membership_period,min,name,price,routine_flg,start_date,年月
0,2019-04-30,CA1,通常,C01,オールタイム,4,10.0,IK166339,NaN,NaT,...,10.0,5.916667,5.0,24.0,3.0,XXXX,10500.0,1.0,2017-04-01,201809
1,2019-04-30,CA1,通常,C01,オールタイム,5,8.0,OA782311,NaN,NaT,...,10.0,6.833333,6.5,15.0,3.0,XXXX,10500.0,1.0,2018-01-01,201902
2,2019-04-30,CA1,通常,C02,デイタイム,4,4.0,HD642174,NaN,NaT,...,8.0,5.333333,5.5,26.0,2.0,XXXX,7500.0,1.0,2017-02-01,201809
3,2019-04-30,CA1,通常,C02,デイタイム,5,5.0,HI859532,NaN,NaT,...,7.0,5.000000,5.0,37.0,2.0,XXX,7500.0,1.0,2016-03-01,201808
4,2019-04-30,CA1,通常,C01,オールタイム,1,3.0,HD459783,NaN,NaT,...,9.0,4.416667,4.5,43.0,1.0,XXXXX,10500.0,1.0,2015-09-01,201812


## ノック44:予測する月の在籍期間を作成

In [64]:
predict_data['now_date'] = pd.to_datetime(predict_data['年月'], format='%Y%m')
predict_data['start_date'] = pd.to_datetime(predict_data['start_date'])

In [66]:
predict_data['period'] = 0
for i in range(len(predict_data)):
    delta = relativedelta(predict_data['now_date'][i], predict_data['start_date'][i])
    predict_data['period'][i] = int(delta.years*12 + delta.months)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [67]:
predict_data.head()

,calc_date,campaign_id,campaign_name,class,class_name,count_0,count_1,customer_id,end_date,exit_date,...,median,membership_period,min,name,price,routine_flg,start_date,年月,now_date,period
0,2019-04-30,CA1,通常,C01,オールタイム,4,10.0,IK166339,NaN,NaT,...,5.0,24.0,3.0,XXXX,10500.0,1.0,2017-04-01,201809,2018-09-01,17
1,2019-04-30,CA1,通常,C01,オールタイム,5,8.0,OA782311,NaN,NaT,...,6.5,15.0,3.0,XXXX,10500.0,1.0,2018-01-01,201902,2019-02-01,13
2,2019-04-30,CA1,通常,C02,デイタイム,4,4.0,HD642174,NaN,NaT,...,5.5,26.0,2.0,XXXX,7500.0,1.0,2017-02-01,201809,2018-09-01,19
3,2019-04-30,CA1,通常,C02,デイタイム,5,5.0,HI859532,NaN,NaT,...,5.0,37.0,2.0,XXX,7500.0,1.0,2016-03-01,201808,2018-08-01,29
4,2019-04-30,CA1,通常,C01,オールタイム,1,3.0,HD459783,NaN,NaT,...,4.5,43.0,1.0,XXXXX,10500.0,1.0,2015-09-01,201812,2018-12-01,39


## ノック45:欠損値を除去

In [68]:
predict_data.isnull().sum()

calc_date               0
campaign_id             0
campaign_name           0
class                   0
class_name              0
count_0                 0
count_1               257
customer_id             0
end_date             2842
exit_date            2842
gender                  0
is_deleted              0
max                     0
mean                    0
median                  0
membership_period       0
min                     0
name                    0
price                   0
routine_flg             0
start_date              0
年月                      0
now_date                0
period                  0
dtype: int64

In [70]:
predict_data = predict_data.dropna(subset=['count_1'])
predict_data.isnull().sum()

calc_date               0
campaign_id             0
campaign_name           0
class                   0
class_name              0
count_0                 0
count_1                 0
customer_id             0
end_date             2637
exit_date            2637
gender                  0
is_deleted              0
max                     0
mean                    0
median                  0
membership_period       0
min                     0
name                    0
price                   0
routine_flg             0
start_date              0
年月                      0
now_date                0
period                  0
dtype: int64

## ノック46:文字列型の変数を処理できるように整形

In [71]:
target_col = ['campaign_name', 'class_name', 'gender', 'count_1', 'routine_flg', 'period', 'is_deleted']
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,通常,オールタイム,M,10.0,1.0,17,0.0
1,通常,オールタイム,M,8.0,1.0,13,0.0
2,通常,デイタイム,F,4.0,1.0,19,0.0
3,通常,デイタイム,F,5.0,1.0,29,0.0
4,通常,オールタイム,M,3.0,1.0,39,0.0


In [73]:
predict_data = pd.get_dummies(predict_data)
del predict_data['campaign_name_通常']
del predict_data['class_name_ナイト']
del predict_data['gender_M']
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
0,10.0,1.0,17,0.0,0,0,1,0,0
1,8.0,1.0,13,0.0,0,0,1,0,0
2,4.0,1.0,19,0.0,0,0,0,1,1
3,5.0,1.0,29,0.0,0,0,0,1,1
4,3.0,1.0,39,0.0,0,0,1,0,0


## ノック47:決定木を用いて退会予測モデルを作成

In [76]:
exit = predict_data.loc[predict_data['is_deleted'] == 1]
conti = predict_data.loc[predict_data['is_deleted'] == 0].sample(len(exit))

In [77]:
X = pd.concat([exit, conti], ignore_index=True)
y = X['is_deleted']
del X['is_deleted']
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [78]:
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)

In [82]:
results_test = pd.DataFrame({'y_test':y_test, 'y_pred':y_test_pred})
results_test.head()

,y_test,y_pred
1296,0.0,0.0
535,1.0,1.0
1059,0.0,0.0
1989,0.0,0.0
1987,0.0,0.0


## ノック48:予測モデルの評価とモデルのチューニング

In [83]:
len(results_test.loc[results_test['y_test'] == results_test['y_pred']]) / len(results_test)

0.8859315589353612

In [84]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.8859315589353612
0.976552598225602


In [85]:
model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best')

In [86]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.908745247148289
0.9302915082382763


## ノック49:モデルに寄与している変数を確認

In [87]:
pd.DataFrame({'Feature_names':X.columns, 'coefficient':model.feature_importances_})

,Feature_names,coefficient
0,count_1,0.374439
1,routine_flg,0.137608
2,period,0.485839
3,campaign_name_入会費半額,0.000000
4,campaign_name_入会費無料,0.000000
5,class_name_オールタイム,0.000000
6,class_name_デイタイム,0.002021
7,gender_F,0.000094


## ノック50:顧客の退会を予測

In [91]:
count_1 = 1
routine_flg = 1
period = 10
campaign_name = "入会費無料"
class_name = "オールタイム"
gender = "M"

In [92]:
if campaign_name == "入会費半額":
    campaign_name_list = [1, 0]
elif campaign_name == "入会費無料":
    campaign_name_list = [0, 1]
elif campaign_name == "通常":
    campaign_name_list = [0, 0]
if class_name == "オールタイム":
    class_name_list = [1, 0]
elif class_name == "デイタイム":
    class_name_list = [0, 1]
elif class_name == "ナイト":
    class_name_list = [0, 0]
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]
input_data = [count_1, routine_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [93]:
print(model.predict([input_data]))
print(model.predict_proba([input_data]))

[1.]
[[0. 1.]]
